# Workbook 11: Depth and Breadth-First Search
## Aims of this practical workbook
1. To give you hands-on experience of configuring depth-first search to try and make it work for the fox-chicken-grain problem
2. To give you hands-on experience of taking code that implements one search method (depth-first) and adapting it to use another (breadth-first)

## Reminder of PseudoCode for Depth-First Search


Variables workingCandidate, openList, closedList

Initialisation: Make initial guess,  test it, then start the openList*    
```
SET workingCandidate = StartSolution
Evaluate (workingCandidate)
IF( IsAtGoal(workingCandidate)) ##lucky guess!
    OUTPUT (SUCCESS, workingCandidate)
APPEND workingCandidate to openList
````

Main loop

    WHILE ( Openlist not empty) DO       ##main search loop ##
        MOVE (last item from openList into working candidate)
        FOREACH (1-step neighbour)
            neighbour = ApplyMoveOperator(workingCandidate) ## Generate
            Evaluate(neighbor)                              ## Test
	        IF(IsAtGoal(neighbour))
                OUTPUT (SUCCESS, neighbour)
            ELSE IF (neighbor is feasible)                  ## Update Memory
                APPEND( neighbor to end of openList)
            ELSE
                APPEND( neighbor to end of closedList) 
        COPY (working candidate to closedList)
 
    OUTPUT (FAILURE, workingCandidate)     ## if no more solutions to test
 

### Example: The fox-chicken-grain problem
- You have a fox, a chicken and a sack of grain.  
- You must cross a river with only one of them at a time.
- If you leave the fox with the chicken he will eat it;
- If you leave the chicken with the grain he will eat it.

Can you get all three across safely in less than ten moves?

## What does generate-and-test look like for fox-chicken-grain problem?

A solution is a sequence of moves of boat with different passengers
```
class candidateSolution:
    def __init__(self):
        self.variableValues = []
        self.quality = 0
        self.depth=0
```

There are 8 moves in total {nothing,fox,chicken,grain} X {bank1to2, bank2to1}
- number these from 0 to 7
- candidateSolution.variableValues is a list of moves

**Evaluate()**: 
score is -1 (infeasible), 0 (ok but doesn't reach goal) or 1 (reaches goal)
- starts from state(0,0,0,0)
- apply move referenced in variableValues[0] to get next state
  - if move can't be applied do nothing and leave state unchanged
  - else if next state in forbidden list return INFEASIBLE (-1)
  - else if next state = (1,1,1,1) return SUCCESS (1)
  - else get and apply next move

Choices for ApplyMoveOperator() on Foreach(1-step neighbour) loop;
- perturbative (use *fixed number of d* moves):  
  nested loop through each position (1...n) and value (0...7) changing  a specific move to the new value
  - i.e. each solution has *d* moves and 7d neighbours (7 different values in d different position)  
  
- constructive:  loop through each possible extra move adding that to the *d* existing ones at depth *d*  
  - i.e.  each solution with *d* moves has  8 neighbours, all with *d+1* moves

<div class = "alert alert-warning" style="color:black">
    <h2> Activity 1: Testing an implementation of depth-first search.</h2>
    <ol>
        <li>Read the code implementation below, then run the two cells below this and  answer the question about what type of search the method is doing</li>
        <li>Make a prediction about whether the code will complete or not. Be honest and write this down (with a reason) <b>before</b> you run the algorithm</li>
        <li> Then run the code and see if your prediction was correct</li>  
        <li> Then answer the two multiple choice questions after the depth-first code cell. <br>
            You may need to alter the value of the variable maxIterations to satisfy yourself about the answers.</li>
    </ol>
</div>

In [1]:
#run this cell to import libraries and utilities
import workbook11_utils as wb11

from workbook11_utils import candidateSolution, TranslateSolutionAsString, Evaluate, IsAtGoal
import copy



In [2]:
#run this cell to display the first question
display(wb11.Q0)

In [3]:
## Common Initialisation

from workbook11_utils import candidateSolution
#Variables workingCandidate, openList, closedList 
workingCandidate = wb11.candidateSolution()
openList = []
closedList = []
reason = ""
## make initial guess,  test it, then start the openList ##
## in this case we start with no moves, depth 0, 
## this does nothing so is not at goal but is feasible
workingCandidate.quality=0
atGoal = False
openList.append(workingCandidate)


##problem specific parts here###
# there are 8 possible next moves: fox/chicken/grain/nothing in boat X  2 directionss
possibleMoves = (0,1,2,3,4,5,6,7)
#Evaluate method was imported above
#IsatGoal was imported above

##search parameters
iteration=1
maxIterations = 50



while( atGoal==False and  len(openList)>0 and iteration<maxIterations): #WHILE ( Openlist not empty) DO
    print(f"Iteration {iteration} there are {len(openList)} candidates on the openList")
    iteration = iteration + 1
    nextItem = len(openList) -1 #MOVE (last item from openList into working candidate)
    workingCandidate = openList.pop(nextItem)
    
    for nextMove in possibleMoves:  #FOREACH (1-step neighbour)       
        ## Generate ##
        neighbour = copy.deepcopy(workingCandidate)     # need to make a deep copy so we can change it 
        neighbour.variableValues.append(nextMove)       #move operator extends current solution
        
        ## Test ## 
        reason = Evaluate(neighbour)
        moveList =TranslateSolutionAsString(neighbour)
        if(IsAtGoal(neighbour)):             #IF AT GOAL OUTPUT (SUCCESS, neighbour)
            print(f'goal found with moves {moveList}')
            atGoal=True
            break ##takes us out of for loop
            
         ## update Working Memory ##
        elif neighbour.quality==0: #ELSE IF (neighbor is feasible)
            print(f'  **adding partial solution: {moveList}')
            openList.append(neighbour) 
        else:
            print(f'    discarding invalid solution: because {reason}')
            closedList.append(neighbour)
 
    ##COPY (working candidate to closedList)
    closedList.append(workingCandidate)

if(atGoal==False):##OUTPUT (FAILURE, workingCandidate)
    print('failed to find solution to the problem in the time allowed!')

Iteration 1 there are 1 candidates on the openList
    discarding invalid solution: because fox eats chicken
    discarding invalid solution: because fox eats chicken
  **adding partial solution: ->Chicken_0to1
    discarding invalid solution: because chicken eats grain
    discarding invalid solution: because boat is in wrong place
    discarding invalid solution: because boat and/or grain is in wrong place
    discarding invalid solution: because  boat and/or chicken is in wrong place
    discarding invalid solution: because  boat and/or fox is in wrong place
Iteration 2 there are 1 candidates on the openList
    discarding invalid solution: because boat is in wrong place
    discarding invalid solution: because boat and/or grain is in wrong place
    discarding invalid solution: because boat and/or chicken is in wrong place
    discarding invalid solution: because  boat and/or fox is in wrong place
  **adding partial solution: ->Chicken_0to1->empty_1to0
    discarding invalid soluti

In [4]:
#run this cell to dispaly the next two questions
display(wb11.Q1)
display(wb11.Q2)

<div class = "alert alert-warning" style="color:black">
    <h2>Activity 2: Restricting the Maximum Depth Search to try and make the algorithm work.</h2>
    The code cell below reproduces the depth-first solution.<br>
    Edit this code so that the maximum depth of the tree search is restricted to some value you can easily change.<br>
    <ol>
        <li> On line 17 define a variable MAXLEN, <b>initially<b> with value 4</li>
        <li> Edit line 48-49, inserting some indented  code (if...else) that changes behaviour depending on the length of the list in the candidateSolution that holds the variable Values ( i.e. on len(neighbour.variableValues) ) <br>
            Make it so that lines which  print the "adding partial solution" message and append the nighbour to the openlist) only happen if that length is less than MAXLEN. <br>
            Otherwise your code should print a message "not adding a neighbour because max depth reached" </li>
    </ol>
            Finally experiment to see what depth (<b>value for MAXLEN</b>)is needed - in other words, how long the sequence of moves has to be.<br>
    Then answer the two multiple choice questions below.
    
</div>

In [5]:
## Common Initialisation

#Variables workingCandidate, openList, closedList 
workingCandidate = wb11.candidateSolution()
openList = []
closedList = []
reason = ""
## make initial guess,  test it, then start the openList ##
## in this case we start with no moves, depth 0, 
## this does nothing so is not at goal but is feasible
workingCandidate.quality=0
atGoal = False
openList.append(workingCandidate)

iteration=1
maxIterations=100



##problem specific parts here###
# there are 8 possible next moves: fox/chicken/grain/nothing in boat X  2 directionss
possibleMoves = (0,1,2,3,4,5,6,7)
#Evaluate method was imported above
#IsatGoal was imported above


while( atGoal==False and  len(openList)>0 and iteration<maxIterations): #WHILE ( Openlist not empty) DO
    print(f"Iteration {iteration} there are {len(openList)} candidates on the openList")
    iteration = iteration + 1
    nextItem = len(openList) -1 #MOVE (last item from openList into working candidate)
    workingCandidate = openList.pop(nextItem)

    for nextMove in possibleMoves:  #FOREACH (1-step neighbour)-constructive        
        ## Generate ##
        neighbour = copy.deepcopy(workingCandidate)         ## need to make a deep copy so we can change it 
        neighbour.variableValues.append(nextMove)       #neighbour = ApplyMoveOperator(workingCandidate)
        
        ## Test ## 
        reason = Evaluate(neighbour)
        moveList =TranslateSolutionAsString(neighbour)
        if(IsAtGoal(neighbour)):             #IF AT GOAL OUTPUT (SUCCESS, neighbour)
            print(f'goal found with moves {moveList}')
            atGoal=True
            break ##takes us out of for loop
            
         ## update Working Memory ##
        elif neighbour.quality==0: #ELSE IF (neighbor is feasible)
            print(f'  **adding partial solution: {moveList}')
            openList.append(neighbour)
        else:
            print(f'    discarding invalid solution: {moveList}  because {reason}')
            closedList.append(neighbour)
 
    ##COPY (working candidate to closedList)
    closedList.append(workingCandidate)

if(atGoal==False):##OUTPUT (FAILURE, workingCandidate)
    print('failed to find solution to the problem in the time allowed!')

Iteration 1 there are 1 candidates on the openList
    discarding invalid solution: ->empty_0to1  because fox eats chicken
    discarding invalid solution: ->Grain_0to1  because fox eats chicken
  **adding partial solution: ->Chicken_0to1
    discarding invalid solution: ->Fox_0to1  because chicken eats grain
    discarding invalid solution: ->empty_1to0  because boat is in wrong place
    discarding invalid solution: ->Grain_1to0  because boat and/or grain is in wrong place
    discarding invalid solution: ->Chicken_1to0  because  boat and/or chicken is in wrong place
    discarding invalid solution: ->Fox_1to0  because  boat and/or fox is in wrong place
Iteration 2 there are 1 candidates on the openList
    discarding invalid solution: ->Chicken_0to1->empty_0to1  because boat is in wrong place
    discarding invalid solution: ->Chicken_0to1->Grain_0to1  because boat and/or grain is in wrong place
    discarding invalid solution: ->Chicken_0to1->Chicken_0to1  because boat and/or chick

In [ ]:
display(wb11.Q3)
display(wb11.Q4)

<div class="alert alert-warning" style="color:black">
    <h2>Activity 3 Convert the depth-first search to breadth-first</h2>
    The code cell below has a copy of the original depth-first search code.<br>
    <ul>
        <li>Edit this to make it implement Breadth-First search.</li>
        <li><b> This should involve changing only one line of code</b></li>
        <li> Then run your code a nd answer the question below</li>
        </ol>
    </div>
    
    

In [ ]:
## Common Initialisation

#Variables workingCandidate, openList, closedList 
workingCandidate = wb11.candidateSolution()
openList = []
closedList = []
reason = ""
## make initial guess,  test it, then start the openList ##
## in this case we start with no moves, depth 0, 
## this does nothing so is not at goal but is feasible
workingCandidate.quality=0
atGoal = False
openList.append(workingCandidate)

iteration=1
maxIterations = 100


##problem specific parts here###
# there are 8 possible next moves: fox/chicken/grain/nothing in boat X  2 directionss
possibleMoves = (0,1,2,3,4,5,6,7)
#Evaluate method was imported above
#IsatGoal was imported above

while( atGoal==False and  len(openList)>0 and iteration<maxIterations): #WHILE ( Openlist not empty) DO
    print(f"Iteration {iteration} there are {len(openList)} candidates on the openList")
    iteration = iteration + 1
    nextItem = len(openList) -1 #MOVE (last item from openList into working candidate)
    workingCandidate = openList.pop(nextItem)
    
    for nextMove in possibleMoves:  #FOREACH (1-step neighbour)       
        ## Generate ##
        neighbour = copy.deepcopy(workingCandidate)         ## need to make a deep copy so we can change it 
        neighbour.variableValues.append(nextMove)       #neighbour = ApplyMoveOperator(workingCandidate)
        
        ## Test ## 
        reason = Evaluate(neighbour)
        moveList =TranslateSolutionAsString(neighbour)
        if(IsAtGoal(neighbour)):             #IF AT GOAL OUTPUT (SUCCESS, neighbour)
            print(f'goal found with moves {moveList}')
            atGoal=True
            break ##takes us out of for loop
            
         ## update Working Memory ##
        elif neighbour.quality==0: #ELSE IF (neighbor is feasible)
            print(f'  **adding partial solution: {moveList}')
            openList.append(neighbour) 
        else:
            print(f'    discarding invalid solution: {moveList} because {reason}')
            closedList.append(neighbour)
 
    ##COPY (working candidate to closedList)
    closedList.append(workingCandidate)

if(atGoal==False):##OUTPUT (FAILURE, workingCandidate)
    print('failed to find solution to the problem in the time allowed!')

In [ ]:
#run this cell to display a question
display(wb11.Q5)

<div class="alert alert-warning" style="color:black">
<h2>Activity 4: IN CLASS TEST</h2>
    
</div>

<div class="alert alert-warning" style="color:black">
<h2>Activity 5 (stretch): Investigate the time and space (memory) requirements of your two methods</h2>
    You should now have working versions of both breadth-first and (restricted) depth-first search. 
    <ol>
        <li> Edit your code to report the total number of solutions examined. <br>
            You could just add a counter that you increment after every time you call the evaluate function <br>
        and then print out the value of the counter at the end.</li>
        <li> Edit your code to create a variable called maxOpenListSize, then  check the length of the openlist at every iteration, and report its maximum size.  </li>
    </ol>
    How do the number of solutions examined and the maximum size of the openlist compare between depth-first and breadth-first search?
    </div>

<div class="alert alert-warning" style="color:black">
<h2>Activity 6 (stretch): change the algorithms above to work in a perturbative fashion with a fixed length of 7 moves</h2>
    This should just involve:
    <ol>
        <li> Changing the initialidation code to some sequence of seven moves. <br>
            e.g. add the line workingCandidate.variableValues = [0,0,0,0,0,0,0] </li>
        <li> instead of one loop that appending 8 moves in turn  to neighbour.variableValues, you will need two loops: one through all the 7 positinos, another through all 8 changes for that position.   </li>
    </ol>
    How do the number of solutions examined and the maximum size of the openlist change?
    </div>

<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>